Timing test suite.

In the following cell: setting up
* imports
* reading data
* defining different functions that we want to time


In [12]:
import numpy as np
import xarray as xr
import scipy as sp
from blockinv_iterative import block_inv, block_iter_inv
from blockinv_iterative_symmetric import block_inv_s


ds_1 = xr.open_dataset("data/regions_verify_isotope_202112_cov.nc")
bio_1 = ds_1["covariance_bio"]
anth_1 = ds_1["covariance_anth"]

ds_2 = xr.open_dataset("data/regions_verify_202104_cov.nc")
bio_2 = ds_2["covariance_bio"]
anth_2 = ds_2["covariance_anth"]

M = bio_2.to_numpy()

print(M == np.transpose(M))

[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


Matrix inv accuracy test

In [13]:
k = int(len(M)/2)

inv1 = np.linalg.inv(M)
inv2 = block_inv(M,k)
#inv3 = block_iter_inv(M, 7)
inv4 = block_inv_s(M, k)

print(np.allclose(inv1, inv2), np.allclose(inv1, inv4))

NameError: name 'np' is not defined

Matrix inv timer

In [ ]:


%timeit np.linalg.inv(M)
%timeit block_inv(M,k)
%timeit block_inv_s(M,k)




6.79 s ± 1.72 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
8.19 s ± 265 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit 2+5


6.08 ns ± 0.053 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)
